In [1]:
import re
import spacy
import pandas as pd
import torch
from torch import nn

df = pd.read_csv("./id.csv", index_col=0)
vocab_dim = df["ID"].max()

nlp = spacy.load("en_core_web_sm")

def search_id(word: str) -> int:
  try:
    return df.loc[[word]]["ID"].values[0]
  except:
    return 0

def tokenizer(title: str):
  words = []

  code_regex = re.compile('[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％]')
  text = code_regex.sub('', title.rstrip().lower())

  doc = nlp(text)

  for token in doc:
    id = search_id(token.text)
    words.append(id)

  return words


# 重み初期化関数の定義
def init_weights(m):
  if isinstance(m, nn.RNN):
    for name, param in m.named_parameters():
      if 'weight_ih' in name:
        nn.init.xavier_uniform_(param.data)
      elif 'weight_hh' in name:
        nn.init.orthogonal_(param.data)
      elif 'bias' in name:
        nn.init.constant_(param.data, 0)

In [2]:
import os
import numpy as np
from dotenv import load_dotenv
from gensim.models import KeyedVectors

load_dotenv()
FILE_DIR = os.getenv('FILE_DIR')
model = KeyedVectors.load_word2vec_format(f"{FILE_DIR}/GoogleNews-vectors-negative300.bin", binary=True)

VOCAB_SIZE = vocab_dim + 2
EMB_SIZE = 300

weights = np.zeros((VOCAB_SIZE, EMB_SIZE))
words_in_pretrained = 0

weights.shape

target = df[df["COUNT"] > 1].index.tolist()

for i, word in enumerate(target):
  try:
    weights[i] = model[word]
    words_in_pretrained += 1
  except KeyError:
    weights[i] = np.random.normal(scale=0.1, size=(EMB_SIZE,))
  
weights = torch.from_numpy(weights.astype((np.float32)))

print(f'学習済みベクトル利用単語数: {words_in_pretrained} / {VOCAB_SIZE}')

学習済みベクトル利用単語数: 6790 / 8195


In [3]:
from torch.utils import data

class NewsDataset(data.Dataset):
  """
  newsのDatasetクラス
  
  Attributes
  ----------------------------
  X : データフレーム
    単語ベクトルの平均をまとめたテンソル
  y : テンソル
    カテゴリをラベル化したテンソル
  phase : 'train' or 'val'
    学習か訓練かを設定する
  """
  
  def __init__(self, X, y, phase='train'):
    self.X = X['TITLE']
    self.y = y
    self.phase = phase
  
  def __len__(self):
    """全データサイズを返す"""
    return len(self.y)
  
  def __getitem__(self, idx):
    """idxに対応するテンソル形式のデータとラベルを取得"""
    inputs = torch.tensor(tokenizer(self.X[idx]))
    return inputs, self.y[idx]

def category_to_label(category: str):
  if 'b' in category:
    return 0
  elif 't' in category:
    return 1
  elif 'e' in category:
    return 2
  elif 'm' in category:
    return 3
  else:
    return -1
  
def collate_fn(batch):
  sequences = [x[0] for x in batch]
  labels = torch.LongTensor([x[1] for x in batch])
  x = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True)
  return x, labels


data_dir = "../chapter06"
train = pd.read_csv(f"{data_dir}/train.txt", sep="\t", header=None, names=['CATEGORY', 'TITLE'])
valid = pd.read_csv(f"{data_dir}/valid.txt", sep="\t", header=None, names=['CATEGORY', 'TITLE'])
test = pd.read_csv(f"{data_dir}/test.txt", sep="\t", header=None, names=['CATEGORY', 'TITLE'])

train_Y = torch.tensor(train["CATEGORY"].map(category_to_label).values)
valid_Y = torch.tensor(valid["CATEGORY"].map(category_to_label).values)
test_Y = torch.tensor(test["CATEGORY"].map(category_to_label).values)

train_dataset = NewsDataset(train, train_Y, phase='train')
valid_dataset = NewsDataset(valid, valid_Y, phase='val')
test_dataset = NewsDataset(test, test_Y, phase='val')

batch_size = 128

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dataloader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_dataloader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

dataloader = {
  "train": train_dataloader,
  "val": valid_dataloader,
  "test": test_dataloader,
}

In [4]:
from torch import optim
from torch.nn import functional as F

class MyCNN(nn.Module):
  def __init__(self, vocab_dim, embedding_dim, padding_idx, output_size, out_channels, kernel_heights, stride, padding, emb_weights):
    super(MyCNN, self).__init__()

    self.embeddings = nn.Embedding.from_pretrained(emb_weights, padding_idx=padding_idx)
    self.conv = nn.Conv2d(1, out_channels, (kernel_heights, embedding_dim), stride, (padding, 0))
    self.drop = nn.Dropout(0.4)
    self.fc = nn.Linear(out_channels, output_size)

  def forward(self, x):
    emb = self.embeddings(x).unsqueeze(1)
    conv = self.conv(emb)
    act = F.relu(conv.squeeze(3))
    max_pool = F.max_pool1d(act, act.size()[2])
    logits = self.fc(self.drop(max_pool.squeeze(2)))
    pred = torch.softmax(logits, dim=-1)
    return pred
  
PADDING_SIZE = vocab_dim + 1
net = MyCNN(VOCAB_SIZE, 300, PADDING_SIZE, 4, 500, 2, 1, 1, weights)
net.apply(init_weights)

net.train()
loss_fn = nn.CrossEntropyLoss()
op = optim.Adam(net.parameters(), lr=0.0005)

In [5]:
from tqdm import tqdm

epochs = 30

for i in range(epochs):
  print("------")
  print(f"Epoch {i}/{epochs}")
  
  for phase in ["train", "val"]:
    epoch_loss = 0.0
    epoch_acc = 0.0
    
    if phase == "train":
      net.train()
    else:
      net.eval()

    for data, label in tqdm(dataloader[phase]):
      op.zero_grad()

      with torch.set_grad_enabled(phase == "train"):
        outputs = net(data)
        loss = loss_fn(outputs, label)
        _, pred = torch.max(outputs, 1)

        if phase == "train":
          loss.backward()
          op.step()

        epoch_loss += loss.item() * data.size(0)
        epoch_acc += torch.sum(pred == label.data)
    
    size = len(dataloader[phase].dataset)
    print(f"train loss: {epoch_loss / size}, acc: {epoch_acc / size}")

------
Epoch 0/30


100%|██████████| 84/84 [01:28<00:00,  1.05s/it]


train loss: 1.232096990232207, acc: 0.5476413369178772


100%|██████████| 11/11 [00:10<00:00,  1.06it/s]


train loss: 1.1454628513244811, acc: 0.6729041934013367
------
Epoch 1/30


...

train loss: 0.9118584016126073, acc: 0.841317355632782
------
Epoch 29/30


100%|██████████| 84/84 [01:27<00:00,  1.04s/it]


train loss: 0.8389262971651327, acc: 0.9155746698379517


100%|██████████| 11/11 [00:09<00:00,  1.14it/s]

train loss: 0.9094903251368128, acc: 0.8368263244628906
